In [ ]:
# Load environment variables
import os
import json 
from typing import List, Dict, Any
from dotenv import load_dotenv

load_dotenv()

# Verify API key is loaded from .env file
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found. Please create a .env file with your API key.")


In [ ]:
pip install llama_index

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.memory import ChatMemoryBuffer


# Configure LLM
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.7)
llm = Settings.llm

In [ ]:
# ANSI color codes for console output styling
class Colors:
    GREEN = "\033[92;1m"
    BLUE = "\033[94;1m"
    YELLOW = "\033[93;1m"
    RED = "\033[91;1m"
    RESET = "\033[0m"

def colored_print(text: str, color: str = Colors.GREEN):
    print(f"{color}{text}{Colors.RESET}")

# Project title
PROJECT_TITLE = "Smart Bookstore Platform for Modern Readers"

# Project Topic/Description (Full Text)
PROJECT_TOPIC = (
    "A web-based mobile app for our bookstore where customers can: "
    "browse books by genre, read previews, purchase books online, "
    "track shipments, review books, and get personalized recommendations."
)

In [ ]:
# Define Agent class
class Agent:
    def __init__(self, name: str, role_desc: str):
        self.name = name
        self.role_desc = role_desc
        self.memory = ChatMemoryBuffer.from_defaults(token_limit=1500)
        self.memory.put(ChatMessage(role=MessageRole.SYSTEM, content=role_desc))

    def chat(self, message: str) -> str:
        self.memory.put(ChatMessage(role=MessageRole.USER, content=message))
        messages = self.memory.get()
        response = llm.chat(messages=messages)
        self.memory.put(response.message)
        return response.message.content

In [ ]:
# Define agent roles
AGENT_ROLES = {
    # Core team roles
    "Product_Owner": "Represents customer needs, manages product backlog, and prioritizes features for the e-commerce platform.",
    "Scrum_Master": "Facilitates Agile ceremonies, removes obstacles, and ensures team adherence to Scrum principles.",
    "UI_UX_Designer": "Designs user interfaces and experiences including wireframes, prototypes, and mobile interfaces.",
    
    # Technical roles
    "Solution_Architect": "Designs system architecture including microservices, integrations, and technical components.",
    "Developer": "Implements features, integrates APIs, and manages frontend/backend development.",
    "DevOps_Engineer": "Manages CI/CD pipelines, infrastructure, and deployment automation.",
    "Security_Engineer": "Conducts security testing, code reviews, and data protection implementations.",
    
    # Quality & Documentation
    "QA_Engineer": "Creates and executes test plans to validate system functionality.",
    "Technical_Writer": "Produces user guides, API documentation, and release notes.",
    
    # Domain specialization
    "Ecommerce_Specialist": "Advises on catalog management, checkout UX, and promotional strategies."
}

# Instantiate agents
agent_instances = {name: Agent(name, desc) for name, desc in AGENT_ROLES.items()}

# Define conversation steps and prompts (Scrum-based for Bookstore App)
CONVERSATION_STEPS = [
    ("Product_Owner", "Scrum_Master"),
    ("Scrum_Master", "UI_UX_Designer"),
    ("UI_UX_Designer", "Scrum_Master"),
    ("Scrum_Master", "Solution_Architect"),
    ("Solution_Architect", "Scrum_Master"),
    ("Scrum_Master", "Developer"),
    ("Developer", "Scrum_Master"),
    ("Scrum_Master", "QA_Engineer"),
    ("QA_Engineer", "Scrum_Master"),
    ("Scrum_Master", "Technical_Writer"),
    ("Technical_Writer", "Scrum_Master"),
]

CONVERSATION_PROMPTS = {
    # Product Owner → Scrum Master
    ("Product_Owner", "Scrum_Master"): (
        "I want to build a web-based mobile app for our bookstore where customers can browse books by genre, read previews, "
        "purchase books online, track shipments, review books, and get personalized recommendations. "
        "Provide your estimate with detailed calculation steps."
    ),
    
    # Scrum Master → UI/UX Designer
    ("Scrum_Master", "UI_UX_Designer"): (
        "Define user stories and acceptance criteria for the bookstore app. "
        "Organize 10+ user stories (e.g., US-01, US-02) with effort estimates. "
        "Show calculation steps."
    ),
    
    # UI/UX Designer → Scrum Master
    ("UI_UX_Designer", "Scrum_Master"): (
        "User stories documented. Calculation steps for effort estimates:"
    ),
    
    # Scrum Master → Solution Architect
    ("Scrum_Master", "Solution_Architect"): (
        "Design the technical architecture for the bookstore app. Prioritize security and scalability. "
        "Estimate components and effort. Show calculations."
    ),
    
    # Solution Architect → Scrum Master
    ("Solution_Architect", "Scrum_Master"): (
        "Architecture design complete. Calculation steps for design effort:"
    ),
    
    # Scrum Master → Developer
    ("Scrum_Master", "Developer"): (
        "Implement features based on user stories and architecture. "
        "Estimate SLOC and effort. Show calculations."
    ),
    
    # Developer → Scrum Master
    ("Developer", "Scrum_Master"): (
        "Development completed. Calculation steps for SLOC effort:"
    ),
    
    # Scrum Master → QA Engineer
    ("Scrum_Master", "QA_Engineer"): (
        "Create and execute test cases for the bookstore app. "
        "Estimate test effort. Show calculations."
    ),
    
    # QA Engineer → Scrum Master
    ("QA_Engineer", "Scrum_Master"): (
        "Testing complete. Calculation steps for test effort:"
    ),
    
    # Scrum Master → Technical Writer
    ("Scrum_Master", "Technical_Writer"): (
        "Prepare user documentation and training materials. "
        "Estimate pages and effort. Show calculations."
    ),
    
    # Technical Writer → Scrum Master
    ("Technical_Writer", "Scrum_Master"): (
        "Documentation complete. Calculation steps for writing effort:"
    ),
}


In [ ]:
# Execute project phase-wise using initiate and response approach
def run_conversation():
    from time import sleep
    colored_print(f"\n{Colors.GREEN}PROJECT TITLE: {PROJECT_TITLE}{Colors.RESET}", Colors.GREEN)
    colored_print(f"\n{Colors.GREEN}PROJECT TOPIC: {PROJECT_TOPIC}{Colors.RESET}", Colors.GREEN)
    colored_print("\nInitiating execution of all project phases...", Colors.BLUE)

    for i, (sender, recipient) in enumerate(CONVERSATION_STEPS):
        prompt = CONVERSATION_PROMPTS.get((sender, recipient), "")
        if not prompt:
            colored_print(f"[STEP {i+1}] {sender} → {recipient}: No prompt defined. Skipping...", Colors.RED)
            continue

        colored_print(f"\n[STEP {i+1}] {sender} → {recipient}", Colors.YELLOW)
        colored_print(f"{sender} says: {prompt}", Colors.GREEN)

        try:
            reply = agent_instances[sender].chat(prompt)
            colored_print(f"{sender} Response to {recipient}:\n{reply}", Colors.BLUE)
        except Exception as e:
            colored_print(f"Error during conversation: {str(e)}", Colors.RED)
            break

        user_input = input("\nType 'exit' to stop or press Enter to continue: ")
        if user_input.strip().lower() == "exit":
            colored_print("\nSimulation exited by user.", Colors.RED)
            break

    colored_print("\n✅ Simulation of all project phases completed successfully.", Colors.GREEN)

# Entry point
if __name__ == "__main__":
    run_conversation()
